In [15]:
import os
os.path.append("../system")

AttributeError: module 'posixpath' has no attribute 'append'

In [14]:
import system.visualize
import image_observers.YOLOv4.detector as yolo
from pathlib import Path
import importlib as imp
import pandas as pd
import seaborn as sns
import bbox
import matplotlib.pyplot as plt
import config.config
import glob
import undistort
from matplotlib.collections import LineCollection
import numpy as np

ModuleNotFoundError: No module named 'sytem'

In [105]:
det = yolo.YOLOv4Detector(cfg_path='image_observers/YOLOv4/yolo4_2306.cfg', weights_path='image_observers/YOLOv4/yolo4_reptilearn260421_best.weights', return_neareast_detection=True)
det.load()

# Analyze video  

In [153]:
dir_path= "/data/reptilearn/experiments/pg20_exp3_20210519_115034"
head_bbox_path= Path(os.path.join(dir_path,'head_bbox.csv'))
head_bbox_df = pd.read_csv(str(head_bbox_path))
bboxes = pd.DataFrame(columns=["x1", "y1", "x2", "y2", "conf"])
vids = glob.glob(dir_path+'/top*mp4')
vid_path = Path(vids[0])
g_lst=[]
vid_path=Path("/data/reptilearn/experiments/pv20_exp3_20210519_133447/top_20210519-133448.mp4")

In [154]:
def detect_frame(orig_frame, write_frame, frame_counter):
    bbox = det.detect_image(orig_frame)
    bboxes.loc[bboxes.shape[0]] = bbox  

In [155]:
def detect_frame(orig_frame, write_frame, frame_counter):
    bbox = det.detect_image(orig_frame)
    g_lst.append(bbox)
    bboxes.loc[bboxes.shape[0]] = bbox   

In [156]:
visualize.process_video(vid_path, None, [detect_frame], start_frame=0, num_frames=None)

  0%|          | 0/128521 [00:00<?, ?it/s]

In [157]:
bboxes[bboxes['x1'].isna()].shape, bboxes.shape

((21548, 5), (128521, 5))

In [158]:
visualize.process_video(vid_path, "/home/ortom/reptilelearn/vid_out.mp4", [
    visualize.offline_centroid_visualizer(bboxes.to_numpy(),color=(255,0,0),window_size=1, dot_radius=10)
    ], num_frames=None)

  0%|          | 0/128521 [00:00<?, ?it/s]

In [143]:
nans=0
poor=0
others=0
none=0
for arr in g_lst:
    try:
        conf=arr[4]
    except:
        none+=1
        others+=1
        continue
    if np.isnan(conf):
        nans+=1
    elif conf<0.8:
        poor+=1
    others+=1
none_sum,none,nans,poor,others

(1229709651, 1846, 0, 33, 97788)

## New weights (26.4.21)
1846/97788 misses on 2nd video (possibly all good because of true negatives)

# Heat map

In [144]:
undist_bboxes = undistort.undistort_data(bboxes, 1440, 1080, config.config.undistort_flir_blackfly_computar)

In [145]:
undist_bboxes

,x1,y1,x2,y2,conf
0,656.377175,873.636989,740.903547,960.468715,0.995870
1,655.989041,873.707133,741.069825,960.802434,0.995900
2,656.017894,873.642061,741.024640,960.733680,0.995898
3,656.110530,873.369300,740.901477,960.589469,0.995850
4,656.302387,873.535999,741.010432,960.751905,0.995901
...,...,...,...,...,...
97783,1271.777978,473.690930,1409.706989,578.085401,0.993151
97784,1273.041793,473.877134,1410.215958,578.422348,0.992790
97785,1270.539984,474.159628,1412.083178,577.267432,0.993250
97786,1271.411771,474.086406,1410.527800,577.578809,0.992131


In [146]:
centroids = bbox.xyxy_to_centroid(undist_bboxes.to_numpy())
cent_df = pd.DataFrame(centroids, columns=["x", "y"])

In [147]:
cent_df

,x,y
0,698.640361,917.052852
1,698.529433,917.254784
2,698.521267,917.187871
3,698.506003,916.979384
4,698.656410,917.143952
...,...,...
97783,1340.742483,525.888165
97784,1341.628875,526.149741
97785,1341.311581,525.713530
97786,1340.969785,525.832607


In [151]:
with open("/home/ortom/reptilelearn/"+os.path.basename(vid_path)+".csv",'w') as newcsv:
    for ls in cent_df.values.tolist():
        ls=[str(i) for i in ls]
        newcsv.write(str(','.join(ls))+'\n')


%matplotlib widget
plt.clf()
ax = sns.histplot(cent_df, x="x", y="y", cbar=True, stat="probability", bins=(32, 27), color="blue")


ax.set_title("Distribution over time")
ax.figure.set_dpi(100)
ax.figure.set_size_inches(9.2,4.8)
#ax.set_xlim(0, 1440)
#ax.set_ylim(0, 1080)
ax.invert_yaxis()
ax.figure.savefig("/home/ortom/reptilelearn/"+os.path.basename(vid_path)+'_1.png')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [102]:
ax.lines

[]

In [93]:
#fig.clf()
fig, ax = plt.subplots(1,1, dpi=100)
ax.add_collection(LineCollection((centroids,), alpha=0.35, linewidths=[1]))
ax.scatter(x=centroids[:, 0], y=centroids[:, 1], s=1, color="red", zorder=100, alpha=0.2)
#ax.set_xlim(0, 1440)
#ax.set_ylim(0, 1080)
ax.invert_yaxis()
ax.set_xlabel("x")
ax.set_ylabel("y")
ax.set_title("Trajectory over time")
ax.set_aspect(1)
ax.figure.savefig("/home/ortom/reptilelearn/"+os.path.basename(dir_path)+'_2.png')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [94]:
ax.get_data()

AttributeError: 'AxesSubplot' object has no attribute 'get_data'

In [160]:
xrange = np.nanmax(centroids[:, 0]) - np.nanmin(centroids[:, 0])
yrange = np.nanmax(centroids[:, 1]) - np.nanmin(centroids[:, 1])

In [172]:
fig.get_size_inches()

array([6.4, 4.8])

# Pose Estimation Video

In [6]:
exp_path = Path("/media/sil2/regev/Pogona_Pursuit/Arena/Archived_Experiments/v1/anti-circle_20201104T142037/trial1/videos/")

In [9]:
vid_path = Path('/media/sil2/regev/Pogona_Pursuit/Arena/Archived_Experiments/v1/anti-circle_20201104T142037/trial1/videos/realtime_20201104T142039.avi')

In [11]:
list((exp_path / "pogona_pursuit_realtime_iteration3").glob("*"))

[PosixPath('/media/sil2/regev/Pogona_Pursuit/Arena/Archived_Experiments/v1/anti-circle_20201104T142037/trial1/videos/pogona_pursuit_realtime_iteration3/realtime.mp4'),
 PosixPath('/media/sil2/regev/Pogona_Pursuit/Arena/Archived_Experiments/v1/anti-circle_20201104T142037/trial1/videos/pogona_pursuit_realtime_iteration3/realtime.csv')]